# Statistical testing for data analysis: a practical guide


## 2. Categorical vs categorical variables




Import libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#!pip install scikit-posthocs
import scikit_posthocs as sp
from statsmodels.graphics.gofplots import qqplot
import scipy.stats as ss
from statsmodels.sandbox.stats.multicomp import multipletests 
from itertools import combinations 

sns.set_theme(style = 'ticks')

## Goal
#### Describe how the characteristics of each patient (e.g., age, sex, and cholesterol levels) affect the metrics of heart function.


Luckily you have access to a Heart Database to do data analysis. Below we describe features included in such database:


- age: Age of the patient

- sex: Sex of the patient

- ex_induced_angina: exercise induced angina (1 = yes; 0 = no)

- major_vessels: number of major vessels colored by fluoroscopy (0-3)

- chestpain: Chest Pain type chest pain type
        Value 1: typical angina
        Value 2: atypical angina
        Value 3: non-anginal pain
        Value 4: asymptomatic
- blood_pressure: resting blood pressure (in mm Hg)

- cholesterol: cholesterol in mg/dl fetched via BMI sensor

- blood_sugar: (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)

- electrcg_results: resting electrocardiographic results
        Value 0: normal
        Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
        Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria
- max_heart_rate: maximum heart rate achieved
- oldpeak: ST depression induced by exercise relative to rest

- chance_heartattack:   0= less chance of heart attack 1= more chance of heart attack

[Source](https://www.kaggle.com/rashikrahmanpritom/heart-attack-analysis-prediction-dataset)

In [ ]:
df = pd.read_csv('https://nycdsaslides.s3.amazonaws.com/LLS/heart.csv',
                 names = ['age','sex','chestpain', 'blood_pressure',\
                          'cholesterol','blood_sugar','electrcg_results',\
                          'max_heart_rate','ex_induced_angina','oldpeak',\
                          'slp','major_vessels','thall','chance_heartattack'], header = 0)

Based on medical evidence, the likelihood of having a heart attack increases after age 45. So let's make a new categorical feature to account for two age groups:

In [ ]:
df.loc[(df['age'] <= 45), 'age_group'] = 'under45'
df.loc[(df['age'] > 45), 'age_group'] = 'over45'

In [ ]:
df.head()

**Question 1** : how does the age group affect the insidence of exercise induced angina?

First, we should present the data in a simple cross tabulation 

In [ ]:
table  = 
table.columns = ['yes','no']
table

## Chi square test

-   $H_0:$ The proportion of patients who have exercise-induced angina is independent of the age group
-   $H_1:$ The proportion of patients who have exercise-induced angina depends on the age group


$$\chi^2 = \sum \frac {(O - E)^2}{E}$$

*O* observed and *E* expected frequencies

The [Yate's correction](https://en.wikipedia.org/wiki/Yates%27s_correction_for_continuity#:~:text=To%20reduce%20the%20error%20in,2%20%C3%97%202%20contingency%20table.) is used when at least one cell of the table has an expected count smaller than 5

| Observed | Yes | No | Total |  |  |  |  |  |  |  |  |  |  |  |  | Expected | Yes    | No    | Total |
|----------|-----|----|-------|--|--|--|--|--|--|--|--|--|--|--|--|----------|--------|-------|-------|
| over45   | 155 | 84 | 239   |  |  |  |  |  |  |  |  |  |  |  |  | over45   | 160.91 | 78.09 | 239   |
| under45  | 49  | 15 | 64    |  |  |  |  |  |  |  |  |  |  |  |  | under45  | 43.09  | 20.91 | 64    |
| **Total**   | 204 | 99 | 303   |  |  |  |  |  |  |  |  |  |  |  |  | **Total**   | 204    | 99    | 303   |

*Expected frequency = row total * column total / grand total* 

In [ ]:
chi2, p_value, dof, expected = 

In [ ]:
expected

In [ ]:
p_value

Since the p-value is > 0.05 que can confidently accpet the null hypothesis that the proportion of patients who have exercise induced angina is independent of the age group

---------
---------

**Question 2** : does the number of major vassels observed by fluorometry depend on the age group?

Again, we should present the data in a cross tabulation 

In [ ]:
table2  = pd.crosstab(,)
table2

In [ ]:
chi2_2, p_value_2, dof_2, ex_2 = ss.chi2_contingency(, correction = )

In [ ]:
p_value_2

Since the p-value is < 0.05 we can confidently reject the null hypothesis and state that the number of major vessels observed by fluorometry does vary depending on the age group.

We're left with the follow-up question of **which groups are significantly different?**




As we did in part 1 of "Statistical testing for data analysis", we can proceed to do pairwise comparisons. However, we need to correct the p-values to control for experimentwise type I error.

One way to do this is to run pair-wise Chi-square tests for the group combinations, and then correct the p-values,  as done with the following functions (adapted from Moran Neuhof's [Chi-square (and post-hoc) tests in Python](https://neuhofmo.github.io/chi-square-and-post-hoc-in-python/) ):

In [ ]:
def run_chisq_on_combination(df, combinations_tuple):
    """Receives a dataframe and a combinations tuple and returns p-value after performing chisq test."""
    assert len(combinations_tuple) == 2, "Combinations tuple is too long! Should be of size 2."
    new_df = df[(df.index == combinations_tuple[0]) | (df.index == combinations_tuple[1])]
    chi2, p, dof, ex = ss.chi2_contingency(new_df, correction=True)
    return p

def chisq_posthoc_corrected(df, correction_method='fdr_bh', alpha=0.05):
    """Receives a dataframe and performs chi2 post hoc tests.
    Prints the p-values and corrected p-values (after FDR correction).
    alpha: optional threshold for rejection (default: 0.05)
    correction_method: method used for mutiple comparisons correction. (default: 'fdr_bh').
    See statsmodels.sandbox.stats.multicomp.multipletests for elaboration."""
    # post-hoc test
    all_combinations = list(combinations(df.index, 2))  # gathering all combinations for post-hoc chi2
    print("Post-hoc chi2 tests results:")
    p_vals = [run_chisq_on_combination(df, comb) for comb in all_combinations]  # a list of all p-values
    # the list is in the same order of all_combinations

    # correction for multiple testing
    reject_list, corrected_p_vals = multipletests(p_vals, method=correction_method, alpha=alpha)[:2]
    for p_val, corr_p_val, reject, comb in zip(p_vals, corrected_p_vals, reject_list, all_combinations):
        print("{}: p_value: {:5f}; corrected: {:5f}, reject H0: {}".format(comb, p_val, corr_p_val, reject))


## 3. Continous vs continous variables


**Question 3** : is there a linear relationship between age and maximum heart rate?

In [ ]:
ax = sns.scatterplot(x = df[''], y = df[''])
ax.set(xlabel = '', ylabel = '');

## Pearson's correlation

For evaluating if the correlation between two numerical or continuous variables is significant.
It is calculated via a Least-Squares fit.

-  1 = perfect positive relationship
-  0 = absence of linear relationship
- -1 = perfect negative relationship

**Assumes two normally distributed continuos variables**

    



-   $H_0:$ There is not a linear relationship in the population
-   $H_1:$ There is a linear relationship in the population

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(13,5))
sns.histplot(df[''], bins = 10, kde = True,
             color = 'purple', ax=ax[0])
sns.histplot(df[''], bins = 10, kde = True,
             color = 'green', ax=ax[1])

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(13,5))
qqplot(df[''], line = , ax = ax[0]) 
qqplot(df[''], line = , ax = ax[1])
fig.show()

In [ ]:
ax = 
sns.regplot(, scatter = False, ci = 95, fit_reg = True, color = 'orange')
ax.set(xlabel = 'Age', ylabel = 'Maximum heart rate');

In [ ]:
R, p_value = 
print(f"Pearson's correlation coefficient : {R:.3f}, p-value: {p_value:.5f}")

The p-value is so small that we reject the null hypothesis and assume that there is a negative relationship between the age of the patients and their maximum heart rate measurement

-------------------
------------------- 



Let's look at an example that's not so well behaved

**Question 4**: is there a linear relationship between the blood pressure and the oldpeak measurement of the patients?

In [ ]:
ax = sns.scatterplot(x = df["blood_pressure"], y = df['oldpeak'])
ax.set(xlabel = 'Blood pressure', ylabel = 'Oldpeak');

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(13,5))
sns.histplot(df['blood_pressure'], bins = 10, kde = True,
             color = 'orange', ax=ax[0])
sns.histplot(df['oldpeak'], bins = 10, kde = True,
             color = 'lightblue', ax=ax[1])

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(13,5))
qqplot(df['blood_pressure'], line = 's', ax = ax[0]) #draw the standardized line with 's' in line argument
qqplot(df['oldpeak'], line = 's', ax = ax[1])
fig.show()

## Spearman's correlation

It is the rank-based equivalent of Pearson's correlation coefficient. It does not assume that the data is normally distributed, and it works well to describe non-linear relationships too ([learn more](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient)).  Also, its use is not only restricted to continuous data but can also be used in analyses of ordinal variables. 


-   $H_0:$ The ranks of one variable do not covary with the ranks of the other variable
-   $H_1:$ The ranks of one variable do covary with the ranks of the other variable

In [ ]:
ax = sns.regplot(x = df["blood_pressure"], y = df['oldpeak'], scatter_kws={'s':8}, fit_reg = False)
sns.regplot(x = df["blood_pressure"], y = df['oldpeak'], scatter = False, ci = 95, fit_reg = True, color = 'orange',  
            lowess = ) 
ax.set(xlabel = 'Blood pressure', ylabel = 'Oldpeak');

In the previous plot we set [lowess]( https://mike-langen.medium.com/creating-powerfull-lowess-graphs-in-python-e0ea7a30b17a#:~:text=LOWESS%20stands%20for%20LOcally%2DWeighted,restricting%20yourself%20to%20any%20form.) = True, which estimates a nonparametric lowess model (locally weighted linear regression)
                                              

In [ ]:
Rs, p_value = 
print(f"Spearman's correlation coefficient : {Rs:.3f}, p-value: {p_value:.5f}")

The p-value is <0.05, so we reject the null hypothesis and assume that the oldpeak and the blood pressure of the patient covary. When one increases, the other one does so too.